# __Computing Minimum-Variance Hedge Ratios__

<br>
<br>

Finance 6470: Derivative Markets <br>
Tyler J. Brough <br>
Last Updated: March 18, 2021 <br>
<br>



## Calculating Sample Statistics for Hedge Ratios

The formula for the optimal static hedge ratio is given in Chapter 3 as:

$$h^{\ast} = \rho \frac{\sigma_{S}}{\sigma_{F}}$$

In practice this will require statistical inference from historical data to **estimate** the values of the paramters: $\rho$, $\sigma_{S}$, and $\sigma_{F}$. 

We will look at how this is to be done.

Recall the formula for the sample standard deviation as the estimator for the standard deviation:

$$\hat{\sigma} = s = \sqrt{\frac{1}{n-1} \sum\limits_{t=1}^{n} (x_{t} - \bar{x})^{2}}$$

where $n$ is the sample size, the $x_{i}$ are the particular historical values of each of the data points for 
$i = 1, \ldots, n$.

Recall also that the estimator for the sample mean is given by:

$$\bar{x} = \frac{1}{n} \sum\limits_{i=t}^{n} x_{t}$$

Below I have reproduced the data in Table 3.2 from the book. We will use these data to calculate the optimal minimum variance hedge ratio.

<br>
<br>

<img src="images/Table3-2.jpg" alt="Cross Hedging" style="width:600px;"/>

<br>
<br>

Lets first compute the sample means for each of the change in futures prices and the change in jet fuel prices.

The mean for the change in heating oil futures prices is:

$$\bar{\Delta F} = \frac{1} {15} (0.021 + 0.035 - 0.046 + \ldots + 0.029) = -0.000867$$

By the way, this can also be easily done in `Python` using the `Numpy` module as:

In [1]:
import numpy as np
from scipy import stats

In [3]:
delta_f = np.array([0.021, 0.035, -0.046, 0.001, 0.044, -0.029, -0.026, -0.029,
                   0.048, -0.006, -0.036, -0.011, 0.019,-0.027, 0.029])

In [4]:
delta_f

array([ 0.021,  0.035, -0.046,  0.001,  0.044, -0.029, -0.026, -0.029,
        0.048, -0.006, -0.036, -0.011,  0.019, -0.027,  0.029])

In [5]:
delta_f.shape

(15,)

In [6]:
fbar = delta_f.mean()

In [7]:
print("The mean of heating oil futures {:f}".format(fbar))

The mean of heating oil futures -0.000867


In [8]:
np.mean(delta_f)

-0.0008666666666666659

Also, the sample standard deviation is:

$$\hat{\sigma}_{\Delta F} = \sqrt{\frac{1}{14} [(0.021 + 0.000867)^2 + (0.035 + 0.000867)^2 + \ldots + (0.029 + 0.000867)^2 ]} = 0.0313$$

Here is how to do it in `Python`:

In [9]:
sigma_f = delta_f.std(ddof=1) # note: use ddof = 1 to get the unbiased estimator!

In [10]:
print("The standard deviation of heating oil futures is {:f}".format(sigma_f))

The standard deviation of heating oil futures is 0.031343


The sample mean for jet fuel spot prices is:

$$\bar{\Delta S} = \frac{1}{15} (0.029 + 0.020 - 0.044 + \ldots + 0.023) = 0.0002$$

Here it is in `Python`:

In [11]:
delta_s = np.array([0.029, 0.020, -0.044, 0.008, 0.026, -0.019, -0.010, -0.007,
                   0.043, 0.011, -0.036, -0.018, 0.009, -0.032, 0.023])

In [12]:
sbar = delta_s.mean()
print("The mean of jet fuel spot prices is: {:f}".format(sbar))

The mean of jet fuel spot prices is: 0.000200


Here is the sample standard deviation for jet fuel spot prices:

$$\hat{\sigma}_{\Delta S} = \sqrt{\frac{1}{14} [(0.029 - 0.0002)^2 + (0.020 - 0.0002)^2 + \ldots + (0.023 - 0.0002)^2 ]} = 0.0263$$

Here is how to do it in `Python`:

In [13]:
sigma_s = delta_s.std(ddof=1)
print("The sample standard deviation of jet fuel prices is: {:f}".format(sigma_s))

The sample standard deviation of jet fuel prices is: 0.026255


## The Sample Correlation Coefficient

We also need to be able to estimate $\rho$ in order to calculate the hedge ratio. Let's recall that the formula for the sample (estimator) correlation coefficient is equal to:

$$\hat{\rho}_{\Delta S, \Delta F} = \frac{\hat{Cov}(\Delta S, \Delta F)}{\hat{\sigma}_{\Delta S} \times \hat{\sigma}_{\Delta F}}$$

So in order to calculate this we need to know how to calculate the sample covariance $\hat{Cov}(\Delta S, \Delta F)$. That formula is:

$$\hat{Cov}(\Delta S, \Delta F) = \hat{\Sigma}_{\Delta S, \Delta F} = \frac{1}{n-1} \sum\limits_{i=1}^{n} (\Delta S_{i} - \bar{\Delta S}) (\Delta F_{i} - \bar{\Delta F})$$

In [14]:
rho = stats.pearsonr(delta_s, delta_f)[0]

In [15]:
h_star = rho * (sigma_s / sigma_f)
print(f"The estimated optimal hedge ratio is: {h_star : 0.4f}")

The estimated optimal hedge ratio is:  0.7777


We can also estimate this value using the estimated slope coefficient in a linear regression.

<br>

Recall that the OLS slope coefficient estimator is equal to the following:

$$
\begin{aligned}
\hat{\beta} &= \frac{Cov(y,x)}{Var(x)} \\
            &= \rho \frac{\sigma_{y} \sigma_{x}}{\sigma_{x} \sigma_{x}} \\
            &= \rho \frac{\sigma_{y}}{\sigma_{x}}
\end{aligned}
$$

<br>

In [16]:
slope, intercept, r_value, p_value, std_err = stats.linregress(delta_f, delta_s)

In [17]:
print(f"The estimated minimum-variance hedge ratio (via OLS) is: {slope : 0.4f}")

The estimated minimum-variance hedge ratio (via OLS) is:  0.7777
